In [0]:
from heapq import heappop, heappush

# `a_star`
#
# `start_chain` - its a class with heuristic function in overload of `__lt__`
# and function `start.last_node()` which return a `node`
# `start_chain` has subclass 'node' with usefull info about graph edge (board state for 15 puzzle)
# and overload of `__eq__` which return result of last move in decision tree.
# `goal` - value which compare with `last_move`; its a target of this algh
#
# `last_node` - must be hashable uniq value
# `heapqpop` working as LIFO if keys are equal
# it means that algh will be worked as deep search in this case


def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

    raise Exception("No solution?!")

In [0]:
import unittest

class test_a_star(unittest.TestCase):
    '''
    This class shows the usage of a_star for other programs
    '''
    class chain():
        """
        Shows which function are be overload for working in a_star algorithm
        """
        def last_node(self):
            """
            Its func must return hashable value (e.g. str)
            """
            return "last_node"
        def f(self):
            """
            Return sum of heuristics
            """
            return 1
        def g(self):
            """
            Return distance from start to self edge
            """
            return 2
        def get_neighbours(self):
            """
            Return neighbours of this edge
            """
            return []
        def __lt__(self, other):
            """
            This is used in priority queue for choosing next proceed object
            """
            return self.f() < other.f()

    def test_usage(self):
        start = self.chain()
        end = start.last_node()
        self.assertEqual(a_star(start, end), start)



In [0]:

from math import sqrt
import argparse

def manh_dst_matrix(a, b, n):
    """Find manhattan distance between `a` and `b` in matrix of size `n`
    """
    return abs(a % n - b % n) + abs(a // n - b // n)


class chain15:
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size ** 2:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == 3:
                sstr += "\n"
            i += 1
        return sstr

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)

    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)

    def last_node(self):
        """Must be hashable value (list not hashable :( )
        """
        return str(self.board_state)

    def linear_conflict(self):
        conflict_count = 0
        # ToDo some heuristic :)
        return 2 * conflict_count

    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    def corner_tiles(self):
        conflict_count = 0
        # upper left corner
        if self.board_state[0] != 1:
            if self.board_state[1] == 2 or self.board_state[self.size] == self.size + 1:
                conflict_count += 1
        # upper right corner
        if self.board_state[self.size - 1] != self.size:
            if self.board_state[self.size - 2] == self.size - 1 or self.board_state[self.size * 2 - 1] == self.size * 2:
                conflict_count += 1
        # lower left corner
        if self.board_state[self.quad_size - self.size] != self.quad_size - self.size + 1:
            if self.board_state[self.quad_size - self.size * 2] == self.quad_size - self.size * 2 + 1 or self.board_state[self.quad_size - self.size + 1] == self.quad_size - self.size + 2:
                conflict_count += 1

        return 2 * conflict_count

    def simple_heur(self):
        dst = 0
        for i in range(0, int(self.quad_size)):
            if (self.board_state[i] - 1) != i:
                dst += 1
        return dst

    def h(self):
        return self.manh_dst() + self.last_move()

    def g(self):
        return len(self.history)

    def f(self):
        return self.h() + self.g()

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # look at neighbours
        if zero_coord + 1 < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord + self.size < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        # # Debug
        # for i in neighs:
        #     print(i.last_node() + "   " + str(i.f()))
        # print("-------------------")
        # time.sleep(1)
        return neighs


if __name__ == '__main__':
    #parser = argparse.ArgumentParser()
    #parser.add_argument("input", help="Input file name")
    #parser.add_argument("output", help="Output file name")
    #args = parser.parse_args()

    board_str = ""

    with open("in.dat") as input_file:
        board_str = input_file.read()

    start_state = list(map(int, board_str.split()))
    start = chain15(start_state)
    end = chain15((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))

    result = a_star(start, end.last_node())
    with open("out.dat", 'w') as output_file:
        print(str(len(result.history)) ,file=output_file)
        for node in result.history:
            print(str(node), file=output_file)
            print("-------------------------", file=output_file)
        print(str(result), file=output_file)